# RUN IN JULIA

Things I did:
1. Add new flow constraint such that in = out for each truck, demand location
2. Add MTZ constraints to avoid cycles within demand locations

Things to try speeding up:
1. Redundant constraints?
2. Better multi-objective form? See the slides... 
3. Put a limit on total number of vehicles
4. Only add feasible arcs? I.e. threshold against edges that are too big of a time in our dataset
5. OPtimize for less things? Don't need hourly wages?

In [1]:
# import Pkg
# Pkg.add("HDF5")

In [2]:
using JuMP
using Gurobi
using CSV
using DataFrames
using HDF5
using Base.Threads


In [3]:
Tx = CSV.read("inputs/Tx.csv", DataFrame, drop=1:1) |> Matrix #N, N
Ty = CSV.read("inputs/Ty.csv",DataFrame, drop=1:1)|> Matrix; #M, N
Tz = CSV.read("inputs/Tz.csv",DataFrame, drop=1:1)|> Matrix; #N, M

M,N = size(Ty)
S = 5

Cf = 1
Cw = 1
Ct = 3
alpha = 0.5
println("M: ", M)
println("N: ", N)
println("S: ", S)

M: 1550
N: 50
S: 5


In [4]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 240)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-05
Set parameter TimeLimit to value 240


In [ ]:

@variable(model, o[1:M], Bin)
@variable(model, x[1:S, 1:N, 1:N], Bin)
@variable(model, y[1:S, 1:M, 1:N], Bin)
@variable(model, z[1:S, 1:N, 1:M], Bin)
@variable(model, f[1:S] >= 0)
@variable(model, u[1:S, 1:N]) #k, j
@variable(model, L >= 0)

@objective(model, Min, alpha*(Cf*sum(o[i] for i in 1:M) + 
Ct * sum(y[k, i, j] for k in 1:S, i in 1:M, j in 1:N) + 
Cw * sum(sum(y[k,i,j]*Ty[i,j] + z[k,j,i]*Tz[j,i] for i in 1:M, j in 1:N) + sum(x[k, j1, j2] * Tx[j1, j2] for j1 in 1:N, j2 in 1:N) for k in 1:S)) +
(1 - alpha) * L)

# @objective(model, Min, 0)


#MTZ Constraitns
for k in 1:S
    for j in 1:N
        @constraint(model, u[k, j] >= 1)
        @constraint(model, u[k, j] <= N)
    end
end

#(MTZ) Between Demand Locations
for j1 in 1:N
    for j2 in 1:N
        if j1 != j2
            for k in 1:S
                @constraint(model, u[k, j2] - u[k, j1] + N * x[k, j1, j2] <= N - 1)
            end
        end
    end
end

# Optional additional constriant
for j in 1:N
    for k in 1:S
        @constraint(model, x[k,j,j] == 0)
    end
end
# end additional constraint

#A truck can leave from at most one factory
for k in 1:S
    @constraint(model, sum(y[k,i,j] for i in 1:M, j in 1:N) <= 1)
end

#Trucks can only travel if they first leave from a factory 
for k in 1:S
    @constraint(model, sum(x[k,j1,j2] for j1 in 1:N, j2 in 1:N) <= (N - 1) * sum(y[k,i,j] for i in 1:M, j in 1:N))
end

#Trucks can only leave used factories
for i in 1:M
    @constraint(model, sum(y[k,i,j] for k in 1:S, j in 1:N) <= S * o[i]) #changed this from N to S
end

#Trucks start and end at the same factory
for k in 1:S
    for i in 1:M
        @constraint(model, sum(z[k,j,i] for j in 1:N) == sum(y[k,i,j] for j in 1:N))
    end
end

#FLOW CONSTRAINTS
for j2 in 1:N #SUM IN = 1
    @constraint(model, sum(sum(y[k,i,j2] for i in 1:M) + sum(x[k,j1,j2] for j1 in 1:N) for k in 1:S) == 1)
end 

for j1 in 1:N  #SUM OUT = 1
    @constraint(model, sum(sum(x[k,j1,j2] for j2 in 1:N) + sum(z[k,j1,i] for i in 1:M) for k in 1:S) == 1)
end 

#If truck k enters location j, it must exit location j
for j in 1:N
    for k in 1:S
        @constraint(model, (sum(y[k,i,j] for i in 1:M) + sum(x[k,j1,j] for j1 in 1:N)) == (sum(x[k,j,j2] for j2 in 1:N) + sum(z[k,j,i] for i in 1:M)) )
    end
end

# Time constraints
for k in 1:S
    @constraint(model, f[k] == sum(y[k,i,j] * Ty[i,j] for i in 1:M, j in 1:N) + sum(x[k,j1,j2] * Tx[j1,j2] for j1 in 1:N, j2 in 1:N))
end

for k in 1:S 
    @constraint(model, f[k] <= L)
end

In [ ]:
optimize!(model)

In [ ]:
try
    rm("results/output.h5")
catch
    1
end
h5write("results/output.h5", "factories", value.(o))
h5write("results/output.h5", "x_edges", value.(x))
h5write("results/output.h5", "y_edges", value.(y))
h5write("results/output.h5", "z_edges", value.(z))
